# Erstellen eines Regressionsmodells: Daten vorbereiten und visualisieren

## **Lineare Regression für Kürbisse - Lektion 2**
#### Einführung

Jetzt, da Sie mit den Werkzeugen ausgestattet sind, die Sie für den Einstieg in den Aufbau von Machine-Learning-Modellen mit Tidymodels und dem Tidyverse benötigen, können Sie beginnen, Fragen an Ihre Daten zu stellen. Wenn Sie mit Daten arbeiten und ML-Lösungen anwenden, ist es äußerst wichtig, zu verstehen, wie man die richtigen Fragen stellt, um das Potenzial Ihres Datensatzes vollständig auszuschöpfen.

In dieser Lektion lernen Sie:

-   Wie Sie Ihre Daten für den Modellaufbau vorbereiten.

-   Wie Sie `ggplot2` für die Datenvisualisierung nutzen.

Die Frage, die Sie beantwortet haben möchten, bestimmt, welche Art von ML-Algorithmen Sie verwenden werden. Und die Qualität der Antwort, die Sie erhalten, hängt stark von der Beschaffenheit Ihrer Daten ab.

Lassen Sie uns dies anhand einer praktischen Übung genauer betrachten.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>


<!--![Kunstwerk von \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.de.jpg)<br>Kunstwerk von \@allison_horst-->


## 1. Importieren von Kürbisdaten und Aufrufen des Tidyverse

Wir benötigen die folgenden Pakete, um diese Lektion zu bearbeiten:

-   `tidyverse`: Das [tidyverse](https://www.tidyverse.org/) ist eine [Sammlung von R-Paketen](https://www.tidyverse.org/packages), die darauf ausgelegt ist, Datenwissenschaft schneller, einfacher und unterhaltsamer zu machen!

Sie können sie wie folgt installieren:

`install.packages(c("tidyverse"))`

Das untenstehende Skript überprüft, ob Sie die für dieses Modul benötigten Pakete haben, und installiert sie für Sie, falls einige fehlen.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Lassen Sie uns nun einige Pakete starten und die [Daten](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) laden, die für diese Lektion bereitgestellt wurden!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Ein schneller `glimpse()` zeigt sofort, dass es Lücken gibt und eine Mischung aus Zeichenketten (`chr`) und numerischen Daten (`dbl`). Das `Date`-Feld ist vom Typ Zeichenkette, und es gibt auch eine seltsame Spalte namens `Package`, in der die Daten eine Mischung aus `sacks`, `bins` und anderen Werten sind. Die Daten sind, ehrlich gesagt, ein bisschen chaotisch 😤.

Tatsächlich ist es nicht sehr häufig, ein Datenset zu erhalten, das vollständig einsatzbereit ist, um direkt ein ML-Modell daraus zu erstellen. Aber keine Sorge, in dieser Lektion wirst du lernen, wie man ein rohes Datenset mit Standardbibliotheken in R vorbereitet 🧑‍🔧. Außerdem wirst du verschiedene Techniken zur Visualisierung der Daten kennenlernen. 📈📊
<br>

> Eine Auffrischung: Der Pipe-Operator (`%>%`) führt Operationen in logischer Reihenfolge aus, indem er ein Objekt an eine Funktion oder einen Ausdruck weiterleitet. Du kannst den Pipe-Operator so verstehen, als würdest du in deinem Code "und dann" sagen.


## 2. Überprüfung auf fehlende Daten

Eines der häufigsten Probleme, mit denen Datenwissenschaftler umgehen müssen, sind unvollständige oder fehlende Daten. R stellt fehlende oder unbekannte Werte mit einem speziellen Platzhalter dar: `NA` (Not Available).

Wie können wir also feststellen, ob der Data Frame fehlende Werte enthält?  
<br>
-   Eine einfache Möglichkeit wäre die Verwendung der Basis-R-Funktion `anyNA`, die die logischen Werte `TRUE` oder `FALSE` zurückgibt.


In [ ]:
pumpkins %>% 
  anyNA()

Super, es scheint, als ob einige Daten fehlen! Das ist ein guter Ausgangspunkt.

-   Eine andere Möglichkeit wäre, die Funktion `is.na()` zu verwenden, die anzeigt, welche einzelnen Spaltenelemente mit einem logischen `TRUE` fehlen.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okay, erledigt, aber bei einem so großen Dataframe wie diesem wäre es ineffizient und praktisch unmöglich, alle Zeilen und Spalten einzeln zu überprüfen😴.

-   Eine intuitivere Methode wäre, die Summe der fehlenden Werte für jede Spalte zu berechnen:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Viel besser! Es fehlen einige Daten, aber vielleicht spielt das für die Aufgabe keine Rolle. Mal sehen, welche weiteren Analysen Ergebnisse bringen.

> Neben den großartigen Paketen und Funktionen verfügt R über eine sehr gute Dokumentation. Zum Beispiel können Sie `help(colSums)` oder `?colSums` verwenden, um mehr über die Funktion zu erfahren.


## 3. Dplyr: Eine Grammatik für Datenmanipulation

<p>
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illustration von @allison_horst</figcaption>


<!--![Illustration von \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.de.png)<br/>Illustration von \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), ein Paket im Tidyverse, ist eine Grammatik für Datenmanipulation, die eine einheitliche Sammlung von Verben bereitstellt, um die häufigsten Herausforderungen bei der Datenmanipulation zu lösen. In diesem Abschnitt werden wir einige der Verben von dplyr erkunden!  
<br>


#### dplyr::select()

`select()` ist eine Funktion aus dem Paket `dplyr`, mit der du Spalten auswählen kannst, die du behalten oder ausschließen möchtest.

Um deinen Data Frame übersichtlicher zu gestalten, kannst du mit `select()` mehrere Spalten entfernen und nur die behalten, die du benötigst.

Zum Beispiel werden wir in dieser Übung die Spalten `Package`, `Low Price`, `High Price` und `Date` für unsere Analyse verwenden. Lass uns diese Spalten auswählen.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` ist eine Funktion im Paket `dplyr`, mit der Sie Spalten erstellen oder ändern können, während die bestehenden Spalten erhalten bleiben.

Die allgemeine Struktur von `mutate` lautet:

`data %>%   mutate(new_column_name = what_it_contains)`

Lassen Sie uns `mutate` ausprobieren, indem wir die Spalte `Date` verwenden und die folgenden Operationen durchführen:

1.  Konvertieren Sie die Daten (derzeit vom Typ Zeichenkette) in ein Monatsformat (es handelt sich um US-Daten, das Format ist also `MM/DD/YYYY`).

2.  Extrahieren Sie den Monat aus den Daten in eine neue Spalte.

Im R-Paket [lubridate](https://lubridate.tidyverse.org/) wird die Arbeit mit Datums- und Zeitdaten erleichtert. Also verwenden wir `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` und schauen, wie wir die oben genannten Ziele erreichen können. Wir können die Spalte `Date` entfernen, da wir sie in den nachfolgenden Operationen nicht mehr benötigen.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Als Nächstes erstellen wir eine neue Spalte `Price`, die den Durchschnittspreis eines Kürbisses darstellt. Jetzt berechnen wir den Durchschnitt der Spalten `Low Price` und `High Price`, um die neue Spalte Price zu füllen.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

„Aber Moment mal!“, wirst du sagen, nachdem du das gesamte Datenset mit `View(pumpkins)` durchgesehen hast, „Hier stimmt doch etwas nicht!“🤔

Wenn du dir die Spalte `Package` ansiehst, werden Kürbisse in vielen verschiedenen Konfigurationen verkauft. Einige werden in `1 1/9 bushel`-Maßen verkauft, andere in `1/2 bushel`-Maßen, einige pro Kürbis, einige pro Pfund und einige in großen Kisten mit unterschiedlichen Breiten.

Lass uns das überprüfen:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Erstaunlich!👏

Kürbisse scheinen sehr schwer konsistent zu wiegen zu sein, daher filtern wir sie, indem wir nur Kürbisse mit dem String *bushel* in der Spalte `Package` auswählen und diese in einen neuen Dataframe `new_pumpkins` speichern.
<br>


#### dplyr::filter() und stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): erstellt eine Teilmenge der Daten, die nur **Zeilen** enthält, die Ihre Bedingungen erfüllen, in diesem Fall Kürbisse mit dem String *bushel* in der Spalte `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): erkennt das Vorhandensein oder Fehlen eines Musters in einem String.

Das [`stringr`](https://github.com/tidyverse/stringr)-Paket bietet einfache Funktionen für gängige String-Operationen.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Du kannst sehen, dass wir uns auf etwa 415 Zeilen Daten beschränkt haben, die Kürbisse in großen Mengen enthalten. 🤩
<br>


#### dplyr::case_when()

**Aber Moment! Es gibt noch etwas zu tun**

Hast du bemerkt, dass die Menge pro Scheffel je nach Zeile variiert? Du musst die Preise normalisieren, sodass sie pro Scheffel angezeigt werden und nicht pro 1 1/9 oder 1/2 Scheffel. Zeit für etwas Mathematik, um das zu standardisieren.

Wir verwenden die Funktion [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), um die Spalte "Price" je nach bestimmten Bedingungen zu *mutieren*. `case_when` ermöglicht es, mehrere `if_else()`-Anweisungen zu vektorisieren.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Jetzt können wir die Preise pro Einheit basierend auf ihrer Buschel-Messung analysieren. All diese Untersuchung der Kürbis-Buschel zeigt jedoch, wie `wichtig` es ist, `die Natur Ihrer Daten zu verstehen`!

> ✅ Laut [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) hängt das Gewicht eines Buschels von der Art des Produkts ab, da es sich um eine Volumenmessung handelt. "Ein Buschel Tomaten soll beispielsweise 56 Pfund wiegen... Blätter und Grünzeug nehmen mehr Platz mit weniger Gewicht ein, sodass ein Buschel Spinat nur 20 Pfund wiegt." Das ist alles ziemlich kompliziert! Lassen wir die Umrechnung von Buschel zu Pfund beiseite und setzen stattdessen den Preis pro Buschel an. All diese Untersuchung der Kürbis-Buschel zeigt jedoch, wie wichtig es ist, die Natur Ihrer Daten zu verstehen!
>
> ✅ Ist Ihnen aufgefallen, dass Kürbisse, die pro halbem Buschel verkauft werden, sehr teuer sind? Können Sie herausfinden, warum? Hinweis: Kleine Kürbisse sind viel teurer als große, wahrscheinlich weil es viel mehr von ihnen pro Buschel gibt, da der ungenutzte Raum von einem großen hohlen Kuchen-Kürbis eingenommen wird.


Nun, zum Schluss, einfach aus Abenteuerlust 💁‍♀️, verschieben wir die Spalte "Month" an die erste Position, also `vor` die Spalte "Package".

`dplyr::relocate()` wird verwendet, um die Position von Spalten zu ändern.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Gute Arbeit! 👌 Du hast jetzt einen sauberen, aufgeräumten Datensatz, mit dem du dein neues Regressionsmodell erstellen kannst!  
<br>


## 4. Datenvisualisierung mit ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografik von Dasani Madipalli</figcaption>


<!--![Infografik von Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.de.png){width="600"}-->

Es gibt ein *weises* Sprichwort, das so lautet:

> "Das einfache Diagramm hat dem Datenanalysten mehr Informationen geliefert als jedes andere Hilfsmittel." --- John Tukey

Ein Teil der Aufgabe eines Data Scientists besteht darin, die Qualität und die Eigenschaften der Daten, mit denen er arbeitet, zu demonstrieren. Dazu erstellen sie oft interessante Visualisierungen, wie Diagramme, Grafiken und Charts, die verschiedene Aspekte der Daten zeigen. Auf diese Weise können sie Beziehungen und Lücken visuell darstellen, die sonst schwer zu erkennen wären.

Visualisierungen können auch dabei helfen, die am besten geeignete Machine-Learning-Technik für die Daten zu bestimmen. Ein Streudiagramm, das einer Linie zu folgen scheint, deutet beispielsweise darauf hin, dass die Daten gut für eine lineare Regression geeignet sind.

R bietet mehrere Systeme zur Erstellung von Grafiken, aber [`ggplot2`](https://ggplot2.tidyverse.org/index.html) ist eines der elegantesten und vielseitigsten. Mit `ggplot2` können Sie Grafiken erstellen, indem Sie **unabhängige Komponenten kombinieren**.

Beginnen wir mit einem einfachen Streudiagramm für die Spalten Price und Month.

In diesem Fall starten wir mit [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), geben einen Datensatz und eine ästhetische Zuordnung (mit [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) an und fügen dann Schichten hinzu (wie [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) für Streudiagramme.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Ist das ein nützlicher Plot 🤷? Überrascht dich irgendetwas daran?

Er ist nicht besonders nützlich, da er lediglich deine Daten als eine Ansammlung von Punkten in einem bestimmten Monat darstellt.
<br>


### **Wie machen wir es nützlich?**

Um Diagramme mit nützlichen Daten anzuzeigen, müssen die Daten normalerweise irgendwie gruppiert werden. In unserem Fall würde beispielsweise das Ermitteln des Durchschnittspreises von Kürbissen für jeden Monat mehr Einblicke in die zugrunde liegenden Muster unserer Daten liefern. Das führt uns zu einem weiteren **dplyr**-Überblick:

#### `dplyr::group_by() %>% summarize()`

Gruppierte Aggregationen in R können einfach berechnet werden mit

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` ändert die Analyseebene von dem gesamten Datensatz zu einzelnen Gruppen, wie beispielsweise pro Monat.

-   `dplyr::summarize()` erstellt einen neuen Dataframe mit einer Spalte für jede Gruppierungsvariable und einer Spalte für jede der angegebenen Zusammenfassungsstatistiken.

Zum Beispiel können wir `dplyr::group_by() %>% summarize()` verwenden, um die Kürbisse basierend auf der **Monat**-Spalte zu gruppieren und dann den **Durchschnittspreis** für jeden Monat zu berechnen.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Knapp und bündig!✨

Kategorische Merkmale wie Monate werden besser mit einem Balkendiagramm 📊 dargestellt. Die Ebenen, die für Balkendiagramme zuständig sind, sind `geom_bar()` und `geom_col()`. Sieh dir `?geom_bar` an, um mehr zu erfahren.

Lass uns eins erstellen!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Dies ist eine nützlichere Datenvisualisierung! Sie scheint darauf hinzudeuten, dass die höchsten Preise für Kürbisse im September und Oktober auftreten. Entspricht das deinen Erwartungen? Warum oder warum nicht?

Herzlichen Glückwunsch zum Abschluss der zweiten Lektion 👏! Du hast deine Daten für den Modellaufbau vorbereitet und anschließend weitere Erkenntnisse mithilfe von Visualisierungen gewonnen!



---

**Haftungsausschluss**:  
Dieses Dokument wurde mithilfe des KI-Übersetzungsdienstes [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
